In [1]:
%pip install tqdm
%pip install plotly
%pip install pandas
%pip install dimcli
%pip install --upgrade pip
%pip install mysql.connector
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [98]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import mysql.connector
from mysql.connector import Error

In [99]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.8
Method: manual login


In [100]:
import json

allNames = []

# Assuming the JSON data is stored in a file named 'data.json'
with open('asci_aap_dataJSON.json') as f:
    data = json.load(f)

people = data["people"]

# Extracting first and last names for each person
for person in people:
    first_name = person["first_name"]
    last_name = person["last_name"]
    names = first_name + " " + last_name
    allNames.append(names.upper())

print(f'There are {len(allNames)} names collected!')

index = allNames.index('R. ALEXANDER')
print(index)

There are 4184 names collected!
61


In [101]:
namesTest = [
    "MICHAEL AGUS",
    "PETER FECCI",
    "TIPPI MACKENZIE",
    "PURNA KASHYAP"]

#Link to grant variables description: https://docs.dimensions.ai/dsl/datasource-grants.html#grants
search_Var = "[abstract + active_year + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + category_sdg + category_uoa + concepts + concepts_scores + date_inserted + dimensions_url + end_date + foa_number + funder_countries + funder_org_acronym + funder_org_cities + funder_org_countries + funder_org_name + funder_org_states + funder_orgs + funders + funding_aud + funding_cad + funding_chf + funding_cny + funding_currency + funding_eur + funding_gbp + funding_jpy + funding_nzd + funding_org_acronym + funding_org_cities + funding_org_city + funding_org_name + funding_org_states + funding_schemes + funding_usd + grant_number + id + investigators + keywords + language + language_title + linkout + original_title + project_numbers + research_org_cities + research_org_countries + research_org_names + research_org_state_codes + research_orgs + researchers + score + start_date + start_year + title]"

In [102]:
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']
    allIds = set()
    for idv in listDict:
        allIds.add(idv['id'])

    allIds = list(allIds)  # Convert set back to a list if needed
    print(f'Total of {len(allIds)} unique ids for {name}: ')
    print(allIds)

    if len(allIds) > 500:
        batched_ids = [allIds[i:i+500] for i in range(0, len(allIds), 500)]
        allWorkingIDs = set()
        
        for batch in batched_ids:
            query_ids = ', '.join([f'"{id_val}"' for id_val in batch])
            query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'
            
            res2 = dsl.query_iterative(query)
            listDict2 = res2.json['researchers']
            
            for person in listDict2:
                if person['obsolete'] == 0:  # Current Working Id(s)
                    allWorkingIDs.add(person["id"])
                else:
                    for ids in person['redirect']:
                        allWorkingIDs.add(ids)
        
        print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
        print(allWorkingIDs)
        
    else:
        query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
        query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'
        
        res2 = dsl.query_iterative(query)
        listDict2 = res2.json['researchers']
        allWorkingIDs = set()
        
        for person in listDict2:
            if person['obsolete'] == 0:  # Current Working Id(s)
                allWorkingIDs.add(person["id"])
            else:
                for ids in person['redirect']:
                    allWorkingIDs.add(ids)
        
        print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
        print(allWorkingIDs)

    
    return allWorkingIDs
#testIDs = researcherIds("MICHAEL AGUS")
#testIDs = researcherIds("R. Alexander")

In [103]:
#using researcher ids to find grants
def findingGrants(ids):
    all_grants = {}
    if len(ids) >= 1:
        for i in ids:
            res = dsl.query_iterative(f"""search grants where researchers = "{i}" return grants{search_Var}""")
            number = res.json['_stats']['total_count']
            print(f'{i} id has {number} grants')
            listDict = res.json['grants']
            name_list = []
            for grant in listDict:
                name_list.append(grant)
            all_grants[i]= name_list
    
    else:
        print(f'+++++++++++There are no working ids+++++++++++')
    return all_grants
        
#findingGrants(testIDs)

In [104]:
database_name = "mit-ps"
column_names = {
    'abstract': 'str',
    'active_year': 'str',
    'category_bra': 'str',
    'category_for': 'str',
    'category_for_2020': 'str',
    'category_hra': 'str',
    'category_hrcs_hc': 'str',
    'category_hrcs_rac': 'str',
    'category_icrp_cso': 'str',
    'category_icrp_ct': 'str',
    'category_rcdc': 'str',
    'category_sdg': 'str',
    'category_uoa': 'str',
    'concepts': 'str',
    'concepts_scores': 'str',
    'date_inserted': 'str',
    'dimensions_url': 'str',
    'end_date': 'str',
    'foa_number': 'str',
    'funder_countries': 'str',
    'funder_org_acronym': 'str',
    'funder_org_cities': 'str',
    'funder_org_countries': 'str',
    'funder_org_name': 'str',
    'funder_org_states': 'str',
    'funder_orgs': 'str',
    'funders': 'str',
    'funding_aud': 'float',
    'funding_cad': 'float',
    'funding_chf': 'float',
    'funding_cny': 'float',
    'funding_currency': 'float',
    'funding_eur': 'float',
    'funding_gbp': 'float',
    'funding_jpy': 'float',
    'funding_nzd': 'float',
    'funding_org_acronym': 'str',
    'funding_org_cities': 'str',
    'funding_org_city': 'float',
    'funding_org_name': 'str',
    'funding_org_states': 'str',
    'funding_schemes': 'str',
    'funding_usd': 'float',
    'grant_number': 'str',
    'id': 'str',
    'investigators': 'str',
    'keywords': 'str',
    'language': 'str',
    'language_title': 'str',
    'linkout': 'str',
    'original_title': 'str',
    'project_numbers': 'str',
    'research_org_cities': 'str',
    'research_org_countries': 'str',
    'research_org_names': 'str',
    'research_org_state_codes': 'str',
    'research_orgs': 'str',
    'researchers': 'str',
    'score': 'float',
    'start_date': 'str',
    'start_year': 'int',
    'title': 'str'
}

In [105]:
def execute_command(query):
    try:
        connection = mysql.connector.connect(host='sql.mit.edu',
                                            database=database_name,
                                            user='mit-ps',
                                            passwd='cut18vuk',
                                            charset='utf8')

        # Check if the connection is established
        if connection.is_connected():
            # print("Connection to database established")

            # Create a cursor and execute the insert statement
            cursor = connection.cursor()
            # print('This is current command: ',query)
            cursor.execute(query)
            connection.commit()  # Commit the transaction
            # print("Data inserted successfully")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

    finally:
        # Close the connection
        if connection.is_connected():
            cursor.close()
            connection.close()
            # print("MySQL connection is closed")


def insert_query(table_name, column, value):
    name = name.replace('-', '_')
    table_name = table_name.replace(' ','_')
    if type(value) == json:
        value = json.dumps(value)
    elif isinstance(value, str):
        value = f"'{value}'"  # Enclose string values in single quotes

    query = f"INSERT INTO {table_name} ({column}) VALUES ({value});"
    return query

In [106]:
 %%time

def dimensionsTrialSearch(name):
    name = name.replace('-', '_')
    table_name = name.replace(' ','_')
    execute_command(f"""
    CREATE TABLE {table_name} (
        abstract TEXT,
        active_year VARCHAR(255),
        category_bra TEXT,
        category_for TEXT,
        category_for_2020 TEXT,
        category_hra TEXT,
        category_hrcs_hc TEXT,
        category_hrcs_rac TEXT,
        category_icrp_cso TEXT,
        category_icrp_ct TEXT,
        category_rcdc TEXT,
        category_sdg TEXT,
        category_uoa TEXT,
        concepts TEXT,
        concepts_scores TEXT,
        date_inserted DATE,
        dimensions_url VARCHAR(255),
        end_date DATE,
        foa_number TEXT,
        funder_countries TEXT,
        funder_org_acronym TEXT,
        funder_org_cities TEXT,
        funder_org_countries TEXT,
        funder_org_name TEXT,
        funder_org_states TEXT,
        funder_orgs TEXT,
        funders TEXT,
        funding_aud FLOAT,
        funding_cad FLOAT,
        funding_chf FLOAT,
        funding_cny FLOAT,
        funding_currency FLOAT,
        funding_eur FLOAT,
        funding_gbp FLOAT,
        funding_jpy FLOAT,
        funding_nzd FLOAT,
        funding_org_acronym TEXT,
        funding_org_cities TEXT,
        funding_org_city FLOAT,
        funding_org_name TEXT,
        funding_org_states TEXT,
        funding_schemes TEXT,
        funding_usd FLOAT,
        grant_number TEXT,
        id TEXT,
        investigators TEXT,
        keywords TEXT,
        language TEXT,
        language_title TEXT,
        linkout TEXT,
        original_title TEXT,
        project_numbers TEXT,
        research_org_cities TEXT,
        research_org_countries TEXT,
        research_org_names TEXT,
        research_org_state_codes TEXT,
        research_orgs TEXT,
        researchers TEXT,
        score FLOAT,
        start_date DATE,
        start_year INT,
        title TEXT
    );
    """) 

    
    nameIds = researcherIds(name)
    if len(nameIds) != 0:
        indvGrants = findingGrants(nameIds)

    else:
        searched = "No ids"
        res = dsl.query_iterative(f"""search grants in investigators for "{name}" return grants{search_Var}""")
        number = res.json['_stats']['total_count']
        typeDict = {}
        print(f'{name} has {number} grants')
        listDict = res.json['grants']
        typeDict[searched] = listDict
        indvGrants = typeDict[searched]
        
    for ids in indvGrants:
        grants = indvGrants[ids]
        for current_grant in grants:
            total_values = []
            for tag in current_grant:
                value = current_grant[tag]
                if not isinstance(value, str):
                    # Convert non-string values to JSON string
                    value = json.dumps(value)

                value = value.replace("'", "\\'")

                # Enclose all values in single quotes
                total_values.append(f"'{value}'")

            columns = ', '.join([f"`{tag}`" for tag in current_grant.keys()])
            values = ', '.join(total_values)
            insert_query = f"INSERT INTO `{table_name}` ({columns}) VALUES ({values});"
            print("SQL Query: ", insert_query)  # Print the query for debugging
            # Execute the query
            execute_command(insert_query)


dimensionsTrialSearch("R. ALEXANDER")


Starting iteration with limit=1000 skip=0 ...


Error: 1142 (42000): CREATE command denied to user 'mit-ps'@'192.42.89.171' for table '_ALEXANDER'


0-550 / 550 (0.75s)
===
Records extracted: 550
Starting iteration with limit=1000 skip=0 ...


Total of 550 unique ids for R. ALEXANDER: 
['ur.01106452566.25', 'ur.07464506527.10', 'ur.01000032157.97', 'ur.015440274061.13', 'ur.012114050170.27', 'ur.07462325070.49', 'ur.012245030554.34', 'ur.070016354.47', 'ur.015443662365.20', 'ur.012550374546.33', 'ur.011603635411.43', 'ur.011075515323.82', 'ur.01101105723.50', 'ur.010513405525.23', 'ur.013571573606.81', 'ur.01103602111.44', 'ur.01336062704.65', 'ur.0703551056.53', 'ur.01215552545.20', 'ur.010240102222.98', 'ur.01005372307.22', 'ur.064547405.13', 'ur.012462645175.84', 'ur.013166062361.43', 'ur.01127470422.16', 'ur.01225027701.25', 'ur.011153620741.63', 'ur.01341060713.33', 'ur.01323232311.39', 'ur.011227106271.87', 'ur.010627607751.08', 'ur.015222201070.29', 'ur.010475235257.60', 'ur.07525053166.64', 'ur.011613776134.73', 'ur.012721553235.31', 'ur.014376543170.06', 'ur.07567262701.66', 'ur.0625517502.21', 'ur.013340477374.22', 'ur.010552034017.67', 'ur.010437763672.82', 'ur.016427665754.33', 'ur.015233410360.21', 'ur.011407205

0-500 / 500 (4.26s)
===
Records extracted: 500
Starting iteration with limit=1000 skip=0 ...
0-50 / 50 (0.30s)
===
Records extracted: 50
Starting iteration with limit=1000 skip=0 ...


...

Error: 1146 (42S02): Table 'mit-ps+DimensionsGrants.R._ALEXANDER' doesn't exist
SQL Query:  INSERT INTO `R._ALEXANDER` (`abstract`, `active_year`, `category_for`, `category_for_2020`, `category_rcdc`, `category_uoa`, `concepts`, `concepts_scores`, `date_inserted`, `dimensions_url`, `end_date`, `foa_number`, `funder_countries`, `funder_org_cities`, `funder_org_countries`, `funder_org_name`, `funder_orgs`, `funders`, `funding_aud`, `funding_cad`, `funding_chf`, `funding_cny`, `funding_currency`, `funding_eur`, `funding_gbp`, `funding_jpy`, `funding_nzd`, `funding_org_city`, `funding_org_name`, `funding_schemes`, `funding_usd`, `grant_number`, `id`, `investigators`, `language`, `language_title`, `linkout`, `original_title`, `project_numbers`, `research_org_cities`, `research_org_countries`, `research_org_names`, `research_orgs`, `researchers`, `score`, `start_date`, `start_year`, `title`) VALUES ('Traditional Mutation Testing produces test cases that distinguish between some description N

Error: 1146 (42S02): Table 'mit-ps+DimensionsGrants.R._ALEXANDER' doesn't exist
SQL Query:  INSERT INTO `R._ALEXANDER` (`abstract`, `active_year`, `category_for`, `category_for_2020`, `category_uoa`, `concepts`, `concepts_scores`, `date_inserted`, `dimensions_url`, `end_date`, `foa_number`, `funder_countries`, `funder_org_cities`, `funder_org_countries`, `funder_org_name`, `funder_orgs`, `funders`, `funding_aud`, `funding_cad`, `funding_chf`, `funding_cny`, `funding_currency`, `funding_eur`, `funding_gbp`, `funding_jpy`, `funding_nzd`, `funding_org_city`, `funding_org_name`, `funding_schemes`, `funding_usd`, `grant_number`, `id`, `investigators`, `language`, `language_title`, `linkout`, `original_title`, `project_numbers`, `research_org_cities`, `research_org_countries`, `research_org_names`, `research_orgs`, `researchers`, `score`, `start_date`, `start_year`, `title`) VALUES ('Abstracts are not currently available in GtR for all funded research. This is normally because the abstract w

Error: 1146 (42S02): Table 'mit-ps+DimensionsGrants.R._ALEXANDER' doesn't exist
SQL Query:  INSERT INTO `R._ALEXANDER` (`abstract`, `active_year`, `category_bra`, `category_for`, `category_for_2020`, `category_hrcs_hc`, `category_rcdc`, `category_uoa`, `concepts`, `concepts_scores`, `date_inserted`, `dimensions_url`, `end_date`, `foa_number`, `funder_countries`, `funder_org_acronym`, `funder_org_cities`, `funder_org_countries`, `funder_org_name`, `funder_orgs`, `funders`, `funding_aud`, `funding_cad`, `funding_chf`, `funding_cny`, `funding_currency`, `funding_eur`, `funding_gbp`, `funding_jpy`, `funding_nzd`, `funding_org_acronym`, `funding_org_city`, `funding_org_name`, `funding_schemes`, `funding_usd`, `grant_number`, `id`, `investigators`, `keywords`, `language`, `language_title`, `linkout`, `original_title`, `project_numbers`, `research_org_cities`, `research_org_countries`, `research_org_names`, `research_orgs`, `researchers`, `score`, `start_date`, `start_year`, `title`) VALUES (

Error: 1146 (42S02): Table 'mit-ps+DimensionsGrants.R._ALEXANDER' doesn't exist
SQL Query:  INSERT INTO `R._ALEXANDER` (`abstract`, `active_year`, `category_bra`, `category_for`, `category_for_2020`, `category_hra`, `category_hrcs_rac`, `category_rcdc`, `category_uoa`, `concepts`, `concepts_scores`, `date_inserted`, `dimensions_url`, `end_date`, `foa_number`, `funder_countries`, `funder_org_acronym`, `funder_org_cities`, `funder_org_countries`, `funder_org_name`, `funder_org_states`, `funder_orgs`, `funders`, `funding_aud`, `funding_cad`, `funding_chf`, `funding_cny`, `funding_currency`, `funding_eur`, `funding_gbp`, `funding_jpy`, `funding_nzd`, `funding_org_acronym`, `funding_org_city`, `funding_org_name`, `funding_org_states`, `funding_schemes`, `funding_usd`, `grant_number`, `id`, `investigators`, `keywords`, `language`, `language_title`, `original_title`, `research_org_cities`, `research_org_countries`, `research_org_names`, `research_org_state_codes`, `research_orgs`, `researcher

SQL Query:  INSERT INTO `R._ALEXANDER` (`abstract`, `active_year`, `category_bra`, `category_for`, `category_for_2020`, `category_hra`, `category_hrcs_hc`, `category_hrcs_rac`, `category_rcdc`, `category_uoa`, `concepts`, `concepts_scores`, `date_inserted`, `dimensions_url`, `end_date`, `foa_number`, `funder_countries`, `funder_org_acronym`, `funder_org_cities`, `funder_org_countries`, `funder_org_name`, `funder_org_states`, `funder_orgs`, `funders`, `funding_aud`, `funding_cad`, `funding_chf`, `funding_cny`, `funding_currency`, `funding_eur`, `funding_gbp`, `funding_jpy`, `funding_nzd`, `funding_org_acronym`, `funding_org_city`, `funding_org_name`, `funding_org_states`, `funding_schemes`, `funding_usd`, `grant_number`, `id`, `investigators`, `language`, `language_title`, `linkout`, `original_title`, `research_org_cities`, `research_org_countries`, `research_org_names`, `research_org_state_codes`, `research_orgs`, `researchers`, `score`, `start_date`, `start_year`, `title`) VALUES ('BA

In [92]:
%%time

def findallPeople(names):
    for name in names[61:]:
        dimensionsTrialSearch(name)
    print(f'Uploaded grants for {len(names)} number of people')
#findallPeople(allNames)


#stopped at R. ALEXANDER